In [1]:
%load_ext autoreload
%autoreload 2

import os

import geovpylib.database as db
import geovpylib.pks as pks
import geovdata.kit as kit


# Siprojuris
pk_project_bhp = 14841 # The pk project in BHP
pk_project_gv = pks.projects.siprojuris # The pk project in GEOVISTORY
metadata_str = 'bhp-add-entity-to-project-siprojuris'

# # Academic Education & Careers (Community Project)
# pk_project_bhp = 14841 # The pk project in BHP
# pk_project_gv = pks.projects.academic_education_and_careers # The pk project in GEOVISTORY
# metadata_str = 'bhp-add-entity-to-project-academic-eduction-and-careers'

# # Monastica
# pk_project_bhp = 14698 # The pk project in BHP
# pk_project_gv = pks.projects.monastica # The pk project in GEOVISTORY
# metadata_str = 'bhp-add-entity-to-project-monastica'

# # Établissements d'enseignement technique
# pk_project_bhp = 15789 # The pk project in BHP
# pk_project_gv = pks.projects.etablissement_enseignement_technique # The pk project in GEOVISTORY
# metadata_str = 'bhp-add-entity-to-project-etab-enseignement-technique'

# # Scholasticon
# pk_project_bhp = 12782 # The pk project in BHP
# pk_project_gv = pks.projects.scholasticon # The pk project in GEOVISTORY
# metadata_str = 'bhp-add-entity-to-project-scholasticon'

# # Espace intellectuel
# pk_project_bhp = 12782 # The pk project in BHP
# pk_project_gv = pks.projects.espace_intellectuel # The pk project in GEOVISTORY
# metadata_str = 'bhp-add-entity-to-project-espace-intellectuel'



# Not to change:
pk_symogih_gv = 6857901 # The pk project of symogih on GEOVISTORY

env = 'prod'
execute = False
import_manner = 'one-shot'

eta = kit.Eta()

# Add entities to project

The goal here is to add a list of entities that were in a particular BHP project, to the equivalent Geovistory project.

## Fetch Geovistory SYMOGIH data

In [2]:
db.connect_geovistory(env, pk_symogih_gv, False, skip_protection=True)

symogih_entities_gv = db.query(f"""
    select
        r.pk_entity as pk_gv,
        a3.string as uri        
    from information.resource r 
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pk_symogih_gv} and ipr.is_in_project = true
    -- URI
    inner join information.statement s1 on s1.fk_subject_info = r.pk_entity and s1.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_symogih_gv} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_object_info and s2.fk_property = {pks.properties.appe_hasValue_string}
    inner join information.appellation a3 on a3.pk_entity = s2.fk_object_info
""")

# Only symogih URIs
symogih_entities_gv = symogih_entities_gv[symogih_entities_gv['uri'].str.contains('symogih.org')]

# Extract PK
symogih_entities_gv['pk_bhp'] = symogih_entities_gv['uri'].str.replace('http://symogih.org/resource/', '', regex=False)
symogih_entities_gv.drop(columns=['uri'], inplace=True)

kit.infos(symogih_entities_gv)

db.disconnect()

# 12s

[DB] Requests will not be executed
[DB] Connecting to PRODUCTION Database ... Connected!
Shape:  (245428, 2) - extract:


,pk_gv,pk_bhp
98,6140930,http://symogih.org/?q=collective-actor-record/...
99,6141350,http://symogih.org/?q=collective-actor-record/...
100,6141622,http://symogih.org/?q=collective-actor-record/...
101,6141764,http://symogih.org/?q=collective-actor-record/219
102,6141871,http://symogih.org/?q=collective-actor-record/...


[DB] Database correctly disconnected.


## Fetch BHP entities

In [3]:
db.connect_external(os.environ.get('YELLOW_BHP'))

symogih_entities_bhp = db.query(f"""
    select 
        fk_collective_actor as pk_bhp_project, fk_object as pk_bhp
    from bhp.associate_project ap
    where ap.fk_collective_actor = {pk_project_bhp}
""")

symogih_entities_bhp = symogih_entities_bhp[['pk_bhp']]

kit.infos(symogih_entities_bhp)

db.disconnect()

# 2s

[DB] Requests will not be executed
[DB] Connecting to PGSQL Database ... Connected!
Shape:  (7726, 1) - extract:


,pk_bhp
0,AcTP31261
1,AcTP31262
2,AcTP31338
3,Info100328
4,AcTP31249


[DB] Database correctly disconnected.


## Filter only the wanted

In [4]:
entities = symogih_entities_gv.merge(symogih_entities_bhp, how='inner')['pk_gv'].tolist()

print('Entity number:', len(entities))

Entity number: 567


## Get all information we have about those entities in Geovistory Symogih project

In [5]:
db.connect_geovistory(env, -1, False, skip_protection=True)

# Get the whole graph ids of those entities (with rules; eg only incoming properties for TeEn)
peits, teens, stmts = db.queries.get_graph_pks(entities, pk_symogih_gv, max_level=100)

# Make the graph explicit
# the_graph = db.explicit_statements(stmts)
# print('Explicit graph:')
# display(the_graph)

# Make a visual graph out of the explicit statements  
# graphs.show(the_graph, 'test.html')

db.disconnect()

# up to 12min

[DB] Requests will not be executed
[DB] Connecting to PRODUCTION Database ... Connected!
> Getting graph of 567 entities (level: 0, )... 21872 statements fetched
  > Getting graph of 8669 entities (level: 1, PeIt)... 16343 statements fetched
    > Getting graph of 5 entities (level: 2, PeIt)... 6 statements fetched
    > Getting graph of 9 entities (level: 2, TeEn)... 18 statements fetched
    > Getting graph of 4 entities (level: 2, OutOfProject)... 4 statements fetched
  > Getting graph of 1703 entities (level: 1, TeEn)... 5500 statements fetched
    > Getting graph of 1537 entities (level: 2, PeIt)... 4981 statements fetched
      > Getting graph of 621 entities (level: 3, PeIt)... 815 statements fetched
      > Getting graph of 892 entities (level: 3, TeEn)... 1827 statements fetched
        > Getting graph of 6 entities (level: 4, PeIt)... 21 statements fetched
          > Getting graph of 6 entities (level: 5, PeIt)... 15 statements fetched
            > Getting graph of 3 entiti

In [6]:
### For info, if needed, to spot unwanted incoming entities

# db.connect_geovistory(env, -1, False, skip_protection=True)
# stmts_expl = db.tools.explicit_statements(stmts)

# display(a.group_by_count(stmts_expl, 'subject')[0:10])
# display(a.group_by_count(stmts_expl, 'property')[0:10])
# display(a.group_by_count(stmts_expl, 'object')[0:10])

## Add those information to the project

In [7]:
db.connect_geovistory(env, pk_project_gv, execute, skip_protection=True)

[DB] Requests will not be executed
[DB] Connecting to PRODUCTION Database ... Connected!


### Before import

In [9]:
ipr_before = db.query(f'select pk_entity, fk_entity from projects.info_proj_rel where fk_project = {pk_project_gv} and is_in_project = true')

print('Number of informations already in the project:', len(ipr_before))
print('Number of informations resulting from the original Symogih project:', (len(entities) + len(peits) + len(teens) + len(stmts) ))

entities_str = db.tools.pks_to_string(entities) # peits + teens)
classes = db.query(f'''
    select 
        r.pk_entity, r.fk_class, ac.dfh_class_label
    from information.resource r
    left join data_for_history.api_class ac on ac.dfh_pk_class = r.fk_class and ac.dfh_class_label_language = 'en'
    where r.pk_entity in {entities_str}
''')
classes['class'] = classes['dfh_class_label'] + ' (' + classes['fk_class'].astype(str) + ')'
classes.drop_duplicates(inplace=True)
print('Classes distribution of the original Symogih project:')
display(kit.group_by_count(classes, 'class'))

Number of informations already in the project: 51501
Number of informations resulting from the original Symogih project: 51924
Classes distribution of the original Symogih project:


,class,count,percent
0,Person (21),566,100.00%


### Filter out already in project

In [10]:
in_project = set(ipr_before['fk_entity'])

entities = list(filter(lambda element: element not in in_project, entities))
peits = list(filter(lambda element: element not in in_project, peits))
teens = list(filter(lambda element: element not in in_project, teens))
stmts = list(filter(lambda element: element not in in_project, stmts))

print('Entities to add to project:', len(entities))
print('PeIts to add to project:', len(peits))
print('TeEns to add to project:', len(teens))
print('Statements to add to project:', len(stmts))

Entities to add to project: 0
PeIts to add to project: 0
TeEns to add to project: 0
Statements to add to project: 0


### Import to the project

In [11]:
db.info_proj_rels.create(entities)
db.info_proj_rels.create(peits)
db.info_proj_rels.create(teens)
db.info_proj_rels.create(stmts)

In [12]:
ipr_after = db.query(f'select pk_entity from projects.info_proj_rel where fk_project = {pk_project_gv} and is_in_project = true')

print('Number of information after import:', len(ipr_after))

Number of information after import: 51501


## Add favorites (take column `ord_num_of_domain` and `ord_num_of_range`)

### Prepare

In [13]:
db.connect_geovistory(env, pk_symogih_gv, execute)

# Fetch all IPR source
ipr_source = db.query(f"""
    select
        pk_entity as ipr_source, fk_entity, fk_project, ord_num_of_domain, ord_num_of_range
    from projects.info_proj_rel
    where fk_project = {pk_symogih_gv} and is_in_project = true
""")
print('IPR source:', len(ipr_source))

# Fetch all IPR target
ipr_target = db.query(f"""
    select
        pk_entity as ipr_target, fk_entity
    from projects.info_proj_rel
    where fk_project = {pk_project_gv} and is_in_project = true
""")
print('IPR target:', len(ipr_target))

# Take only the ones we are interested in
iprs = ipr_target.merge(ipr_source)

# Format & prepare request
iprs_to_reset = iprs.ipr_target.unique().tolist()
iprs_to_1 = iprs[iprs.ord_num_of_domain == 1].ipr_target.unique().tolist()

print('IPRs to reset:', len(iprs_to_reset))
print('IPRs to set to 1:', len(iprs_to_1))

# 12s

[DB] Requests will not be executed
[DB] Connecting to PRODUCTION Database ... Connected!
IPR source: 3608947
IPR target: 51501
IPRs to reset: 51497
IPRs to set to 1: 990


### Execute

In [15]:
iprs_to_reset_str = db.tools.pks_to_string(iprs_to_reset)
iprs_to_1_str = db.tools.pks_to_string(iprs_to_1)

In [16]:
db.execute(f"""
    update projects.info_proj_rel
        set ord_num_of_domain = NULL
    where pk_entity in {iprs_to_reset_str} and ord_num_of_domain != NULL and pk_entity not in {iprs_to_1_str};
""")

[DB] SQL not executed


In [17]:
db.execute(f"""
    update projects.info_proj_rel
        set ord_num_of_domain = 1
    where pk_entity in {iprs_to_1_str} and ord_num_of_domain != 1;
""")

[DB] SQL not executed


### Finish

In [18]:
db.disconnect()

[DB] Database correctly disconnected.
